In [4]:
import os
import csv
import dateparser
import re
import json
from typing import Union, Tuple
from py_pdf_parser import loaders
from py_pdf_parser.components import PDFDocument, PDFPage, PDFElement
from datetime import datetime

In [4]:
reader = loaders.load_file(r"JOs\joe_20210321_0069_c000.pdf")

In [5]:
page = reader.get_page(3)

In [6]:
if len(page.elements.filter_by_text_contains("Décrète")) > 0:
    before = page.elements.before(page.elements.filter_by_text_contains("Décrète").extract_single_element())
    raw = page.elements.after(page.elements.filter_by_text_contains("Décrète").extract_single_element())
else:
    before = []
    raw = page.elements

In [7]:
a.get_decrees_count(reader)

3

In [8]:
reader.number_of_pages

142

In [9]:
mega_string = ""

In [10]:
for i in range(2,reader.number_of_pages+1):
    mega_string = mega_string+" "+" ".join([re.sub(r"\s+|\t+|\n"," ",ele.text()).strip() for ele in reader.get_page(i).elements]).split("JOURNAL  OFFICIEL  DE  LA  RÉPUBLIQUE  FRANÇAISE ")[-1]

In [11]:
pattern = re.compile(r"([A-Z\-]*\s\(([a-z-A-Z\s\,\-À-ÿ\’]*){1,9}\)\,)(.*?)(\,\sdpt\s[0-9]{2,3}|\,\sdép\.\s[0-9]{2,3}){1}", re.UNICODE)

In [12]:
last_bit = "Naturalisations et réintégrations Décret du 26 février 2021 rapportant un décret de naturalisation NOR : INTN2100355D Par décret en date du 26 février 2021, sur l’avis conforme du Conseil d’Etat, est rapporté le décret du 14 février 2018 (Journal officiel du 16 février 2018) portant naturalisation de: Mme NGAMALEU TIAKO Béatrice née le 1er mars 1979 à Douala (Cameroun), 2017X025985."

In [13]:
mini_string = mega_string[0:5177] + last_bit

In [14]:
mini_string_old = "ABACHICH (Lakbira), née le 31/12/1946 à Tasdermt (Maroc), NAT, 2020X 014555, dép. 006, Dt. 006/1. ABADA (Aymen), né le 03/02/1985 à Sousse (Tunisie), NAT, 2020X 014556, dép. 006, Dt. 006/2. ABADA (Selima), née le 28/05/2014 à Nice (06300), EFF, 2020X 014556, dép. 006, Dt. 006/3. ABADA (Soulaymen, Sadak), né le 10/05/2018 à Nice (06300), EFF, 2020X 014556, dép. 006 AARIF (Sonia), au lieu de AARIF (Sounia), née le 01-05-2006, à Oujda (Maroc), Dt. FR 602/1 - 2020F 000365, dpt 075 AIT-YOUCEF (Aldjia, Françoise), au lieu de AIT-YOUCEF (Aldjia), née le 11-06-1984, à Azazga (Algérie), Dt. FR 602/2 - 2020F 000150, dpt 99 ALTIN (Céline), au lieu de ALTIN (Rojin), née le 27-12-2007, à Nevers (58000), Dt. FR 602/6 - 2021F 000095, dpt 058 UPON, et afin que (DYLAN) s’appelle COLLIN SENGA-ZAKOS OU DUVAL SENGA-ZAKOS OU ou DUPON. 21 mars 2021 Texte 109 JOURNAL OFFICIEL DE LA RÉPUBLIQUE FRANÇAISE ANNONCES Les annonces sont reçues à la direction de l’information légale et administrative Demandes de changement de nom : téléprocédure sécurisée Fiche pratique disponible sur https://psl.service-public.fr/mademarche/pub-changement-nom/demarche Autres annonces : annonces.jorf@dila.gouv.fr ou DILA, DIRE JOURNAUX OFFICIELS, TSA No 71641, 75901 PARIS CEDEX 15 (L'Administration décline toute responsabilité quant à la teneur des annonces.) DEMANDES DE CHANGEMENT DE NOM No 01959 Mme BEN-TAYEB (VIRGINIE, CATHY, GAELLE), née le 7 août 1985 à NEMOURS, 77140, FRANCE, demeurant APPT 2506, BATIMENT 11, 25 RUE DES GRANGES, MORET SUR LOING, MORET LOING ET ORVANNE, 77250, dépose une requête auprès du garde des sceaux afin de s’appeler à l’avenir ZENHAKER. ISSN 0373-0425 La Directrice de l’information légale et administrative : Anne DUCLOS-GRISIER"

In [15]:
mini_string

' 21 mars 2021 Texte 69 JOURNAL OFFICIEL DE LA RÉPUBLIQUE FRANÇAISE Naturalisations et réintégrations Décret du 19 mars 2021 portant naturalisation, réintégration, mention d’enfants mineurs bénéficiant de l’effet collectif attaché à l’acquisition de la nationalité française par leurs parents et francisation de noms et de prénoms NOR : INTN2104968D Le Premier ministre, Sur le rapport du ministre de l’intérieur, Vu le code civil, notamment ses articles 17 à 32 ; Vu la loi no 72-964 du 25 octobre 1972 modifiée relative à la francisation des noms et prénoms des personnes qui acquièrent, recouvrent ou se font reconnaître la nationalité française, Décrète : Art. 1er. – Sont naturalisés Français, réintégrés dans la nationalité française et saisis par l’effet collectif attaché à l’acquisition de la nationalité française par leurs parents les étrangers dont les noms suivent : ABACHICH (Lakbira), née le 31/12/1946 à Tasdermt (Maroc), NAT, 2020X 014555, dép. 006, Dt. 006/1. ABADA (Aymen), né le 0

In [16]:
pattern.search(mini_string).group()

'ABACHICH (Lakbira), née le 31/12/1946 à Tasdermt (Maroc), NAT, 2020X 014555, dép. 006'

In [17]:
pattern.search(mini_string).end()

964

In [18]:
final = []
i=0
while i<= re.search(r"rapportant un décret de naturalisation|portant francisation de noms", mega_string).start():
    if pattern.search(mega_string,pos=i) is not None:
        found = pattern.search(mega_string,pos=i).group()
        i = pattern.search(mega_string,pos=i).end()
        final.append(found)
    else: break


In [58]:
serie = "010"
for person in final:
    if f"), NAT, 2020X {serie}" in person:
        print(person)
        #print(person.split(f"), NAT, 2020X {serie}")[-1].split(",")[0])

ABRAHAMYAN (Enrik), né le 09/04/1996 à Erévan (Arménie), NAT, 2020X 010977, dép. 057
ADJAI (Adélohunkè, Lise-Line), née le 29/08/1990 à Cotonou (Bénin), NAT, 2020X 010059, dép. 091
AHDJOUDJ (Karim), né le 24/06/1970 à Rouiba (Algérie), NAT, 2020X 010215, dép. 075
KARAMI (Mohammad Amin), né le 18/03/1998 à Kermanshah (Iran), NAT, 2020X 010661, dép. 091
ALIA (Soundes, Hibet, Rahmen), née le 21/04/2000 à Khenchela (Algérie), NAT, 2020X 010695, dép. 092
ANOAR (Shahrazad), née le 08/04/1989 à Casablanca (Maroc), NAT, 2020X 010656, dép. 092
BALET (Johan, Ouriel), né le 12/01/1994 à Brazzaville (République du Congo), NAT, 2020X 010885, dép. 068
BASAID (Zahra), née le 28/05/1992 à Casablanca (Maroc), NAT, 2020X 010693, dép. 092
BOTHA (Corlia, Leoné), née le 09/05/1983 à Sandton (Afrique du sud), NAT, 2020X 010171, dép. 075
BOUSSAHA (Malek), né le 09/08/1991 à Tabarka (Tunisie), NAT, 2020X 010816, dép. 074
BOUZNIF (Khaled), né le 05/12/1992 à Medenine (Tunisie), NAT, 2020X 010837, dép. 091
CAMP

In [47]:
print(final[0])

"), NAT, 2020X 014" in final[0]

ABACHICH (Lakbira), née le 31/12/1946 à Tasdermt (Maroc), NAT, 2020X 014555, dép. 006


True

In [1]:
from reader import Reader

In [2]:
a = Reader()

1
4
5
7
10
22
28
32
32
34
37
45
214


In [3]:
len([lolo for lolo,val in a.naturalized.items() if val])

214

In [3]:
a.read_pdf("JOs\joe_20210502_0103_c000.pdf")

Here
re.compile('(Décret\\sdu)(.*?)(\\sNOR){1}')
Finished first page
Décret modificatif du
100 824110


In [5]:
reader = loaders.load_file("JOs\joe_20210502_0103_c000.pdf")
page_last = reader.page_numbers[-1]
pattern_person = re.compile(r"([A-Z\-]*\s\(([a-z-A-Z\s\,\-À-ÿ\’]*){1,9}\)\,)(.*?)(\,\sdpt\s[0-9]{2,3}|\,\sdép\.\s[0-9]{2,3}){1}", re.UNICODE)
pattern_first = re.compile(r"(Décret\sdu)(.*?)(\sNOR){1}", re.UNICODE)
print(pattern_first)
first_page=" ".join([re.sub(r"\s+|\t+|\n"," ",ele.text()).strip() for ele in reader.get_page(1).elements]).split("JOURNAL  OFFICIEL  DE  LA  RÉPUBLIQUE  FRANÇAISE ")[-1]
print("Finished first page")

re.compile('(Décret\\sdu)(.*?)(\\sNOR){1}')
Finished first page


In [7]:
if "Décret modificatif du" in first_page:
    if re.search(r"Décret modificatif du",first_page).start() > re.search(r"portant naturalisation",first_page).start():
        pattern_last = r"Décret modificatif du"
    else:
        if "Annonces" in first_page:
            pattern_last = r"Les annonces sont reçues à la direction de l’information légale et administrative"
        else:
            pattern_last = r"ISSN\s[0-9]*\-{0,1}[0-9]*"
elif "rapportant un décret de naturalisation" in first_page:
    pattern_last = r"rapportant un décret de naturalisation"
elif "Annonces" in first_page:
    pattern_last = r"Les annonces sont reçues à la direction de l’information légale et administrative"
else:
    pattern_last = r"ISSN\s[0-9]*\-{0,1}[0-9]*"
del first_page
mega_string = ""
for i in range(2,page_last+1):
    mega_string = mega_string+" "+" ".join([re.sub(r"\s+|\t+|\n"," ",ele.text()).strip() for ele in reader.get_page(i).elements]).split("JOURNAL  OFFICIEL  DE  LA  RÉPUBLIQUE  FRANÇAISE ")[-1]
del reader
final = []
print(pattern_last)
i = re.search(pattern_first, mega_string).start()
final_pos = re.search(pattern_last, mega_string).start()
print(i,final_pos)
mega_string = mega_string[i:final_pos]
del final_pos,i,pattern_first,pattern_last

Décret modificatif du
100 824110


In [13]:
final = ["".join(ele) for ele in pattern_person.findall(mega_string)]

In [15]:
final[-1]

'DOMINE (Steve, Alain), né le 16/04/1963 à Genève (Suisse), LIB, 2021X 005300, dép. 99'